This script retrieves the list of luftdaten sensors that are currently delivering data.
It then filter the sensors list on a fiven boundary (bbox) and store the list into an sqlite database.

In [10]:
import pandas as pd
import requests
import config
import sqlalchemy

In [2]:
engine = sqlalchemy.create_engine(f"sqlite:///{config.DB_PATH}")

In [11]:
# all currently active sensors on lufdaten with location,sensor type:
URL_24h =  "https://maps.luftdaten.info/data/v2/data.24h.json"

In [12]:
r = requests.get(URL_24h).json()
df = pd.DataFrame(r)
df_geo_location=df.location.apply(pd.Series).add_prefix('location_')
df = pd.concat([df,df_geo_location],axis=1)

In [13]:
df_sensor_type = df.sensor.apply(pd.Series).sensor_type.apply(pd.Series).add_prefix('sensor_type_')
df_sensor_id = df.sensor.apply(pd.Series)[['id']].add_prefix('sensor_')

In [14]:
df = pd.concat([df,df_sensor_id,df_sensor_type],axis=1)

In [15]:
df.drop(['location','sensor','sensordatavalues','id','sampling_rate','timestamp'],axis=1,inplace=True)

In [8]:
df.to_sql(name='sensors_luftdaten', con=engine,if_exists='replace')

In [9]:
df.sensor_type_name.unique()

array(['SDS011', 'PPD42NS', 'DHT22', 'BME280', 'HTU21D', 'BMP180',
       'SDS021', 'PMS7003', 'HPM', 'DS18B20', 'BMP280', 'PMS5003',
       'DS18S20', 'SHT31', 'PMS1003', 'PMS3003', 'Laerm'], dtype=object)